In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [3]:
transform=transforms.Compose([
    transforms.Resize((200,200)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [4]:
train_data = ImageFolder(root='data/Train/', transform=transform)
train_loader=torch.utils.data.DataLoader(train_data,batch_size=16,shuffle=True,num_workers=2)

In [5]:
class_names=['AI', 'Real']

In [6]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1=nn.Conv2d(3,12,5) 
        self.pool=nn.MaxPool2d(2,2)  
        self.conv2=nn.Conv2d(12,24,5) 
        self.conv3=nn.Conv2d(24,48,5)
        self.conv4=nn.Conv2d(48,96,5)
        self.fc1=nn.Linear(96*8*8,2000)
        self.fc2=nn.Linear(2000,1000)
        self.fc3=nn.Linear(1000,500)
        self.fc4=nn.Linear(500,256)
        self.fc5=nn.Linear(256,64)
        self.fc6=nn.Linear(64,16)
        self.fc7=nn.Linear(16,2)

    def forward(self, x):
       x=self.pool(F.relu(self.conv1(x)))
       x=self.pool(F.relu(self.conv2(x)))
       x=self.pool(F.relu(self.conv3(x)))
       x=self.pool(F.relu(self.conv4(x)))
       x=torch.flatten(x,1)
       x=F.relu(self.fc1(x))
       x=F.relu(self.fc2(x))
       x=F.relu(self.fc3(x))
       x=F.relu(self.fc4(x))
       x=F.relu(self.fc5(x))
       x=F.relu(self.fc6(x))
       x=self.fc7(x)
       return x


In [7]:
net=NeuralNet().to(device)
loss_function=nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(),lr=0.001, weight_decay=1e-5)

In [8]:
for epoch in range(15):
    print(f'Training epoch {epoch}..')

    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    print(f'Epoch {epoch + 1}: Loss = {epoch_loss:.4f}, Accuracy = {epoch_accuracy:.2f}%')

Training epoch 0..
Epoch 1: Loss = 0.5476, Accuracy = 76.40%
Training epoch 1..
Epoch 2: Loss = 0.4301, Accuracy = 82.59%
Training epoch 2..
Epoch 3: Loss = 0.3789, Accuracy = 84.53%
Training epoch 3..
Epoch 4: Loss = 0.3109, Accuracy = 87.12%
Training epoch 4..
Epoch 5: Loss = 0.2653, Accuracy = 90.79%
Training epoch 5..
Epoch 6: Loss = 0.1618, Accuracy = 94.68%
Training epoch 6..
Epoch 7: Loss = 0.1609, Accuracy = 94.82%
Training epoch 7..
Epoch 8: Loss = 0.0661, Accuracy = 97.99%
Training epoch 8..
Epoch 9: Loss = 0.0612, Accuracy = 98.27%
Training epoch 9..
Epoch 10: Loss = 0.0400, Accuracy = 98.56%
Training epoch 10..
Epoch 11: Loss = 0.1522, Accuracy = 97.55%
Training epoch 11..
Epoch 12: Loss = 0.0430, Accuracy = 98.56%
Training epoch 12..
Epoch 13: Loss = 0.0539, Accuracy = 98.42%
Training epoch 13..
Epoch 14: Loss = 0.0434, Accuracy = 98.78%
Training epoch 14..
Epoch 15: Loss = 0.0637, Accuracy = 98.85%


In [9]:
torch.save(net.state_dict(),'trained_net.pth')

In [10]:
net = NeuralNet().to(device)
net.load_state_dict(torch.load('trained_net.pth'))

C:\Users\srini\AppData\Local\Temp\ipykernel_6232\617875171.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [11]:
import os
import csv
from PIL import Image

def load_image(path, image_path):
    full_path = os.path.join(path, image_path)
    image = Image.open(full_path).convert("RGB")
    image = transform(image)
    image = image.unsqueeze(0).to(device)  
    return image

# Paths and configurations
path = 'data/Test/'
output_csv_path = 'predictions.csv'

image_paths = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
image_paths.sort()  

predictions = []
net.eval()
with torch.no_grad():
    for img_path in image_paths:
        image = load_image(path, img_path)
        output = net(image)
        _, predicted = torch.max(output, 1)
        predictions.append((img_path[:-4], class_names[predicted.item()]))

with open(output_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Id', 'Label'])  
    writer.writerows(predictions)

print(f"Predictions saved to {output_csv_path}")

Predictions saved to predictions.csv
